# 📓  05_data_transformation.ipynb (Scaling & Encoding)

In [1]:
# # Data Transformation
# Scaling and encoding for modeling


# ## 1. Import Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

print("🔄 Starting data transformation...")

🔄 Starting data transformation...


In [2]:
# ## 2. Load Data with Features
df = pd.read_csv('../data/processed/fraud_data_with_features.csv')
print(f"📊 Dataset shape: {df.shape}")

# Separate features and target
X = df.drop('class', axis=1)
y = df['class']

print(f"📋 Features shape: {X.shape}")
print(f"🎯 Target shape: {y.shape}")

📊 Dataset shape: (129146, 48)
📋 Features shape: (129146, 47)
🎯 Target shape: (129146,)


In [3]:
# ## 3. Identify Column Types
print("🔍 Identifying column types...")

# Get column data types
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
datetime_cols = X.select_dtypes(include=['datetime64']).columns.tolist()

print(f"🔢 Numeric columns ({len(numeric_cols)}):")
print(numeric_cols[:10], "..." if len(numeric_cols) > 10 else "")

print(f"\n🔤 Categorical columns ({len(categorical_cols)}):")
print(categorical_cols)

print(f"\n🕒 Datetime columns ({len(datetime_cols)}):")
print(datetime_cols)

🔍 Identifying column types...
🔢 Numeric columns (37):
['user_id', 'purchase_value', 'age', 'ip_address', 'purchase_hour', 'ip_address_int', 'lower_bound', 'upper_bound', 'is_high_risk_country', 'time_since_signup_hours'] ...

🔤 Categorical columns (10):
['signup_time', 'purchase_time', 'device_id', 'source', 'browser', 'sex', 'purchase_day', 'country', 'purchase_day_name', 'age_group']

🕒 Datetime columns (0):
[]


In [4]:
# ## 4. Drop Unnecessary Columns
print("\n🗑️  Dropping unnecessary columns...")

# Columns to drop (IDs, raw IPs, exact timestamps)
cols_to_drop = [
    'user_id', 'device_id', 'ip_address', 'ip_address_int',
    'lower_bound', 'upper_bound', 'country',
    'signup_time', 'purchase_time', 'prev_purchase_time',
    'purchase_day_name', 'age_group'
]

# Only drop if they exist
existing_cols_to_drop = [col for col in cols_to_drop if col in X.columns]
X = X.drop(existing_cols_to_drop, axis=1)

print(f"Dropped {len(existing_cols_to_drop)} columns:")
for col in existing_cols_to_drop:
    print(f"  - {col}")

# Update column lists after dropping
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

print(f"\n📊 After dropping: {X.shape}")


🗑️  Dropping unnecessary columns...
Dropped 12 columns:
  - user_id
  - device_id
  - ip_address
  - ip_address_int
  - lower_bound
  - upper_bound
  - country
  - signup_time
  - purchase_time
  - prev_purchase_time
  - purchase_day_name
  - age_group

📊 After dropping: (129146, 35)


In [5]:
# ## 5. Handle Categorical Variables
print("\n🔤 Encoding categorical variables...")

# Create a copy for Label Encoding (simpler approach)
X_encoded = X.copy()

# Apply Label Encoding to categorical columns
label_encoders = {}
for col in categorical_cols:
    if col in X_encoded.columns:
        # Handle NaN values first
        X_encoded[col] = X_encoded[col].fillna('Unknown')
        
        # Apply label encoding
        le = LabelEncoder()
        X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
        label_encoders[col] = le
        
        print(f"  ✓ Encoded: {col} ({len(le.classes_)} unique values)")

print(f"\n✅ Encoded {len(label_encoders)} categorical columns")



🔤 Encoding categorical variables...
  ✓ Encoded: source (3 unique values)
  ✓ Encoded: browser (5 unique values)
  ✓ Encoded: sex (2 unique values)
  ✓ Encoded: purchase_day (7 unique values)

✅ Encoded 4 categorical columns


In [6]:
# ## 6. Scale Numerical Features
print("\n📏 Scaling numerical features...")

# Identify numerical columns after encoding
numeric_cols_final = X_encoded.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Remove target-related columns if they exist
numeric_cols_final = [col for col in numeric_cols_final if col != 'class']

print(f"Scaling {len(numeric_cols_final)} numerical columns")

# Apply StandardScaler (good for most algorithms)
scaler = StandardScaler()
X_scaled = X_encoded.copy()
X_scaled[numeric_cols_final] = scaler.fit_transform(X_encoded[numeric_cols_final])

print("✅ Numerical features scaled")


📏 Scaling numerical features...
Scaling 35 numerical columns
✅ Numerical features scaled


In [7]:
# ## 7. Feature Correlation Check
print("\n🔗 Checking feature correlations...")

# Calculate correlation with target
correlations = pd.DataFrame({
    'feature': X_scaled.columns,
    'correlation_with_target': [np.corrcoef(X_scaled[col], y)[0, 1] 
                                for col in X_scaled.columns]
}).sort_values('correlation_with_target', key=abs, ascending=False)

print("\n📊 Top 10 features by absolute correlation:")
print(correlations.head(10))

# Check for highly correlated features (potential multicollinearity)
correlation_matrix = X_scaled.corr().abs()
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
highly_correlated = [column for column in upper_triangle.columns 
                     if any(upper_triangle[column] > 0.8)]

if highly_correlated:
    print(f"\n⚠️  Highly correlated features (>0.8): {highly_correlated}")
else:
    print("\n✅ No highly correlated features found")


🔗 Checking feature correlations...

📊 Top 10 features by absolute correlation:
                    feature  correlation_with_target
25         new_customer_1hr                 0.717107
30        suspicious_device                 0.681906
28         device_frequency                 0.671768
29  unique_users_per_device                 0.671768
26        new_customer_24hr                 0.666534
27       new_customer_7days                 0.485032
10           purchase_month                -0.313517
8   time_since_signup_hours                -0.261308
11    purchase_day_of_month                -0.163499
7      is_high_risk_country                 0.046667

⚠️  Highly correlated features (>0.8): ['avg_purchase_per_user', 'new_customer_24hr', 'device_frequency', 'unique_users_per_device', 'suspicious_device', 'high_risk_browser']


In [8]:
# ## 8. Split Data
print("\n✂️  Splitting data into train and test sets...")

from sklearn.model_selection import train_test_split

# Split with stratification (preserve fraud ratio)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, 
    test_size=0.2, 
    stratify=y,
    random_state=42
)

print(f"✅ Data split completed:")
print(f"   Training set: {X_train.shape}")
print(f"   Test set: {X_test.shape}")
print(f"   Training fraud rate: {y_train.mean():.4%}")
print(f"   Test fraud rate: {y_test.mean():.4%}")



✂️  Splitting data into train and test sets...
✅ Data split completed:
   Training set: (103316, 35)
   Test set: (25830, 35)
   Training fraud rate: 9.4990%
   Test fraud rate: 9.5006%


In [9]:
# ## 9. Save Transformed Data
print("\n💾 Saving transformed data...")

import joblib

# Save all components
transformation_data = {
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test,
    'feature_names': X_scaled.columns.tolist(),
    'label_encoders': label_encoders,
    'scaler': scaler,
    'numeric_cols': numeric_cols_final,
    'categorical_cols': categorical_cols
}

joblib.dump(transformation_data, '../data/processed/transformed_data.pkl')

# Also save as CSV for easy inspection
X_train_with_target = pd.concat([X_train, y_train.reset_index(drop=True)], axis=1)
X_test_with_target = pd.concat([X_test, y_test.reset_index(drop=True)], axis=1)

X_train_with_target.to_csv('../data/processed/X_train_transformed.csv', index=False)
X_test_with_target.to_csv('../data/processed/X_test_transformed.csv', index=False)

print("✅ Data transformation completed!")
print(f"\n📁 Saved files:")
print("   - transformed_data.pkl")
print("   - X_train_transformed.csv")
print("   - X_test_transformed.csv")
print(f"\n📊 Final feature count: {X_train.shape[1]}")


💾 Saving transformed data...
✅ Data transformation completed!

📁 Saved files:
   - transformed_data.pkl
   - X_train_transformed.csv
   - X_test_transformed.csv

📊 Final feature count: 35
